In [2]:
import scipy as sp
import matplotlib.pyplot as plt
import autograd.numpy as np

from scipy.optimize import minimize
from autograd import grad

import scipy.sparse as sp
import scipy.sparse.linalg as spl

## Newton Intro
Actual Newton methods for nonlinear programming problems typically involves an update of a set of optimizatio parameters $x_k$ that looks like:
\begin{equation}
    x_{k+1} = x_k + H_k^{-1}\nabla f(x_k)
\end{equation}
where $H_k$ is the hessian of the objective and $\nabla$ is the Jacobian.

The class of quasi-newton methods are small: SR1, BFGS, DRP, Broyden class (all of which are related)

## L-BFGS
This is probably the state of the art quasi newton. Others include conjugate gradients.
\begin{equation}
    B_{k+1} = B_k + \frac{y_ky_k^T}{y_k^Ts_k} - \frac{B_ks_ks_k^TB_k^T}{s_k^TB_ks_k}
\end{equation}
where $B_k$ is the "approximate" Hessian. Even for this "approximate" Hessian, the size alone is prohibitive, so a limited memory version of this involves:

rank one matrix: one linearly independent row or column (i.e. a scalar is rank one)
rank two matrix: two linearly independent rows or columns? (i.e. 2x2 identity amtrix is rank two)


## LBFGS (limited) memory
So, can we tell what in the BFGS algorithm costs a lot? It's Hk, most likely. LBFGS overcomes his by approximating $Hk$ through it's diagonal only. The algorithm we attempt to implement will be based off the Wikipedia description, which uses a two loop recursion.

THe LBFGS requires us to store multiple gradients seen sequentially during the optimization (so it doesn't start until we've seen k gradients for example)

So, the L-BFGS algorithm is just a new way to compute this approximation:

$z_k$ is what is used now to denote the final search direction.

\begin{equation}
    q= g_k
\end{equation}

We further define:
\begin{equation}
s_k = x_{k+1}-x_k
\end{equation}

\begin{equation}
y_k = g_{k+1}-g_k
\end{equation}

I would interpret $s_k$ as a first order difference of the variable and $s_k$ as a first order difference of the gradient.

where $g_k$ is the gradient computed a the $kth$ 

for $ i = k-1, k-2, ... k-m $

\begin{equation}
    \alpha_i = \rho_i s_i^Tq
\end{equation}
where $\rho_i$ is given by:
\begin{equation}
    \rho_i = \frac{1}{s_i^T y_i}
\end{equation}
\begin{equation}
    q = q-\alpha_iy_i
\end{equation}



Once we finish this loop to update our $q$, then we compute
\begin{equation}
    \gamma_k = \frac{s_{k-1}^{T}y_{k-1}}{s_k^Ty_{k-1}}
\end{equation}
and we set our first approximation of the Hessian

\begin{equation}
    H^{(0)}_k = \gamma_k I
\end{equation}

We are at iteration $k$ and we have $s_k$, which requires $x_{k+1}$


Then we finish with one for for loop:

for $ i = k-1, k-2, ... k-m $
\begin{equation}
    \beta_i = \rho_iy_i^Tz
\end{equation}

\begin{equation}
    z = z+ s_i(\alpha_i - \beta_i)
\end{equation}


## Wolfe Searches


### Strong Wolfe
The strong wolfe conditions appear to require a re-evaluation of the GRADIENT, but it does seem to minimize, in general the number of evaluations.


In [4]:
## Use the Rosenbrock function
import autograd

def rosen2(arr):
    return ((1 - arr[0])**2 + 100*(arr[1] - arr[0]**2)**2) 
    
#grad_rosen2 = grad(rosen2)

print(rosen2(np.array([1,2])))

rosenbrock_with_grad = autograd.value_and_grad(rosen2)

rosen2_grad = grad(rosen2);

print(rosenbrock_with_grad(np.array([1.,2.])))

## minimum is 1, 1
print(rosen2(np.array([1.,1.])), rosen2_grad(np.array([0.,0.])))

print(rosen2(np.array([0.05865546, 0.00233125])))


100
(100.0, array([-400.,  200.]))
0.0 [-2.  0.]
0.8862525783330446


In [5]:
## multidimensional rosenbrock function

def rosen( x ):  # rosen.m
        # a sum of squares, so LevMar (scipy.optimize.leastsq) is pretty good
    x0 = x[:-1]
    x1 = x[1:]
    return (np.sum( (1 - x0) **2 )
        + 100 * np.sum( (x1 - x0**2) **2 ))

print(rosen(np.random.rand(10)))

158.20305849752


The general rosenbrock is:
\begin{equation}
f(\mathbf{x}) = \sum_{i}^{N}(100(x_{i+1}-x_i^2)^2 +(1-x_i^2)
\end{equation}

The 2D function has a gradient:
\begin{equation}
    \nabla f(x,y) = (-200(y-x^2)x - 2x, 200(y-x^2))
\end{equation}

In [6]:
## other functions
def ellipse( x ):
    return np.mean( (1 - x) **2 )  + 100 * np.mean( np.diff(x) **2 )

#...............................................................................
def nesterov( x ):
    """ Nesterov's nonsmooth Chebyshev-Rosenbrock function, Overton 2011 variant 2 """
    x0 = x[:-1]
    x1 = x[1:]
    return np.abs( 1 - x[0] ) / 4 \
        + np.sum( np.abs( x1 - 2*abs(x0) + 1 ))

#...............................................................................
def saddle( x ):
    return np.mean( np.diff( x **2 )) \
        + .5 * np.mean( x **4 )

In [7]:
f = saddle;
gradf = grad(saddle)

In [12]:
##LBFGS with obj and grad_j provided by autograd
from collections import deque
ITERS = 10;
m = 10; #number of gks and sks that we store


n = 1000; #dimension of n-dimensional rosenbrock

x0 = np.zeros(n)
x0 = np.random.rand(n)
g0 = np.zeros(n)
H0 = sp.identity(n); #first approximation of the hessian
history = deque(maxlen = m);


gradient_history = deque(maxlen = m)
sk_history = deque(maxlen = m)
yk_history = deque(maxlen = m)

Sm = np.zeros((n,m))
Ym = np.zeros((n,m))
## outer index i

gradToler = 1e-10; #% tolerance for the norm of the slope
XToler = 1e-10;    #% tolerance for the variables' refinement

## RUN 1 ITERATION PRIOR to entering the loop
# assumes function is differentiable
[f0,g0]=f(x0), gradf(x0); # f0 is evaluation, g0 is the gradient
                        # we want to move the gradient eval as a seperate function
#print(f0, g0)
    
# line search
# usually line search method only return step size alpha
# we return 3 variables to save caculation time.
[alpha,f1,g1, eval_count] = strongwolfe(f, gradf,-g0,x0,f0,g0);
print('initial ls count: ', eval_count)
#print(alpha, f1, g1)
x1 = x0 - alpha*g0;
k =0;

for _ in range(ITERS):
    
    ## use norm tolerance...this prevents the algorithm from having divide by zero errors
    fnorm = np.linalg.norm(g0);
    if(fnorm < gradToler):
        break;
    
    s0 = x1-x0; #nx1
    y0 = g1-g0; #nxn
    
    ## should this be a vector or not a vector?
    gamma_k = np.dot(s0.T,y0)/np.dot(y0,y0)
    hdiag = gamma_k*sp.identity(n); #diagonal of the hessian, dneominator is scalar, numerator is a vector
    p = np.zeros((len(g0),1));
    #print(hdiag)
    ## UPDATE SEARCH DIRECTION
    if(k<m):  ##Cache isn't full
        # update S,Y
        Sm[:,k] = s0;
        Ym[:,k] = y0;
        # never forget the minus sign
        ## this doesn't seem to recompute any new g0 
        p = -getHg_lbfgs(g1,Sm[:,:k],Ym[:,:k],hdiag); 
        
    elif(k>=m): # the cache is full, reupdate the cache
        Sm[:,:m-1]=Sm[:,1:m];
        Ym[:,:m-1]=Ym[:,1:m];
        Sm[:,m-1] = s0;
        Ym[:,m-1] = y0;    
        # never forget the minus sign
        p = -getHg_lbfgs(g1,Sm,Ym,hdiag);
   
    ## LINE SEARCH:
    [alpha ,fs,gs, eval_count]= strongwolfe(f, gradf,p,x1,f1,g1);
    print('evals for line search: ', eval_count)
    x0 = x1; ## update x0 with x1 the delta
    g0 = g1;
    
    ## make a step
    x1 = x1 + alpha*p;
    f1 = fs;
    g1 = gs;
    # save caculation
    # [f1,g1]=feval(myFx,x1);
    k = k + 1;
    
print(f1, f(x1))

initial ls count:  4
evals for line search:  1
evals for line search:  1
evals for line search:  1
evals for line search:  1
evals for line search:  1
evals for line search:  1
evals for line search:  1
evals for line search:  1
evals for line search:  1
evals for line search:  1
-0.00047876426688119614 -0.00047876426688119614


  (0, 0)	396.6967682980991
  (1, 1)	396.6967682980991
  (2, 2)	396.6967682980991
  (3, 3)	396.6967682980991
  (4, 4)	396.6967682980991
  (5, 5)	396.6967682980991
  (6, 6)	396.6967682980991
  (7, 7)	396.6967682980991
  (8, 8)	396.6967682980991
  (9, 9)	396.6967682980991
  (10, 10)	396.6967682980991
  (11, 11)	396.6967682980991
  (12, 12)	396.6967682980991
  (13, 13)	396.6967682980991
  (14, 14)	396.6967682980991
  (15, 15)	396.6967682980991
  (16, 16)	396.6967682980991
  (17, 17)	396.6967682980991
  (18, 18)	396.6967682980991
  (19, 19)	396.6967682980991
  (20, 20)	396.6967682980991
  (21, 21)	396.6967682980991
  (22, 22)	396.6967682980991
  (23, 23)	396.6967682980991
  (24, 24)	396.6967682980991
  :	:
  (975, 975)	396.6967682980991
  (976, 976)	396.6967682980991
  (977, 977)	396.6967682980991
  (978, 978)	396.6967682980991
  (979, 979)	396.6967682980991
  (980, 980)	396.6967682980991
  (981, 981)	396.6967682980991
  (982, 982)	396.6967682980991
  (983, 983)	396.6967682980991
  (984, 98

In [117]:
#print(hdiag, s0,y0, np.outer(s0, y0))

In [9]:
def getHg_lbfgs(g,S,Y,hdiag):
    '''
        GIVES US APPROXIMATION OF HESSIAN we need to determine next search direction
        Input
           S:    Memory matrix (n by k) , s{i}=x{i+1}-x{i}
           Y:    Memory matrix (n by k) , df{i}=df{i+1}-df{i}
           g:    gradient (n by 1)
           hdiag value of initial Hessian diagonal elements (scalar)
        Output
           Hg    the the approximate inverse Hessian multiplied by the gradient g    
    '''
    n,k = S.shape;
    ro = np.zeros(k);
    for i in range(k):
        ro[i] = 1/np.dot(Y[:,i],S[:,i]);
    
    q = np.zeros((n,k+1)); #
    r = np.zeros(n);   #
    
    ## 
    alpha = np.zeros(k);
    beta = np.zeros(k);
    
    ##step 1
    q[:,k] = g;

    ## first loop
    for i in range(k-1, -1, -1):
        alpha[i] = ro[i]*np.dot(S[:,i].T, q[:,i+1]);
        q[:,i] = q[:,i+1]-alpha[i]*Y[:,i];

    ## Multiply by Initial Hessian
    ## r should be a vector
    r = hdiag@q[:,0];

    ## second loop
    for i in range(k):
        beta[i] = ro[i]*np.dot(Y[:,i].T,r);
        r = r + S[:,i]*(alpha[i]-beta[i]);
    
    Hg=r;
    return Hg;



   

## WOLFE SEARCH INFRASTRUCTURE

In [10]:
def strongwolfe(f,gradf, d, x0, fx0, gx0):
    '''
        f: function 
        gradf: gradient of function (autograd)
        d: search direction
        x0: initial start
        fx0: initial function value at x0
        gx0: initial gradient at x0; should be a vector
        
        OUTPUTS:
        alphas: step size
        fs:     the function value at x0+alphas*d
        gs:     the gradient value at x0+alphas*d  
    '''
    ## only up to 3 iterations allowed?
    
    maxIter = 3; # how many iterations to run 
    alpham = 20;
    alphap = 0;
    c1 = 1e-4;
    c2 = 0.9;
    
    alphax = 1;
    gx0 = np.dot(gx0.T,d); ## what's this 
    fxp = fx0;
    gxp = gx0;
    i=1;
    eval_count = 0;
    while(True):
        xx = x0 +alphax*d; #proposed new val;
        ## evaluate function and gradient
        fxx, gxx = f(xx), gradf(xx);
        eval_count+=1;
        fs = fxx;
        gs = gxx;
        gxx = np.dot(gxx.T,d); ## project on the direction of search?
        if(fxx > fx0 + c1*alphax*gx0 or ((i>1) and (fxx >= fxp))): #evaluate wolfe condition;
            [alphas,fs,gs, subc] = Zoom(f,gradf,x0,d,alphap,alphax,fx0,gx0);
            eval_count+=subc
            break;
        if(np.abs(gxx) <= -c2*gx0): #not sure what this condition is
            alphas = alphax;
            break;
        if(gxx >= 0):
            [alphas,fs,gs, subc] = Zoom(f,gradf,x0,d,alphax,alphap,fx0,gx0);
            eval_count+=subc

            break;
        ## all breaking conditions were not satisfied
        alphap = alphax;
        fxp = fxx;
        gxp = gxx;
        if(i > maxIter):
            alphas = alphax;
            break
        ## r = rand(1);%randomly choose alphax from interval (alphap,alpham)
        r = 0.8;
        alphax = alphax + (alpham-alphax)*r;
        i+=1;
    
    return alphas, fs, gs, eval_count
            
def Zoom(f, gradf, x0,d,alphal,alphah,fx0,gx0):
    '''
        myFx:
        x0:
        d:
        alphal:
        alphah:
        fx0:
        gx0:
        
        returns
            alphas
            fs
            gs 
            eval_counts
        
    '''
    c1 = 1e-4;
    c2 = 0.9;
    i =0;
    maxIter = 3
    while(True):
        ## bisection
        alphax = 0.5*(alphal+alphah);
        alphas = alphax;
        xx = x0 + alphax*d;
        ## this is highly inefficient if we need to evaluate grad
        [fxx,gxx] = f(xx), gradf(xx);
        fs = fxx;
        gs = gxx;
        gxx = np.dot(gxx.T,d);
        xl = x0 + alphal*d;
        
        fxl = f(xl); #no gradient required
        
        if((fxx > fx0 + c1*alphax*gx0) or (fxx >= fxl)):
            alphah = alphax;
        else:
            if(abs(gxx) <= -c2*gx0):
                alphas = alphax;
                break;
            if(gxx*(alphah-alphal) >= 0):
                alphah = alphal;
            alphal = alphax;
       
        i = i+1;
        if(i > maxIter):
            alphas = alphax;
        
    return alphas,fs,gs, i
    
        
        
        